In [1]:
%%capture
!pip install accelerate==0.26.1 peft==0.7.1 bitsandbytes==0.42.0 transformers==4.35.2 trl==0.7.10 datasets==2.16.1 evaluate
!pip uninstall wandb -y

In [2]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data preparation

In [3]:
data = load_dataset('dair-ai/emotion', 'split')
idx2label = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
label2idx = {v:e for e, v in enumerate(idx2label)}
data['train'][0]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated', 'label': 0}

In [4]:
PROMPT = "Identify the sentiment in the text:"

def prepare_data_chatml_format(sample):
    """
    <|im_start|>user
    {query} <|im_end|>
    <|im_start|>assistant
    {response} <|im_end|>
    """
    sample['text'] = f"<|im_start|>user\n{PROMPT} {sample['text']} <|im_end|>\n<|im_start|>assistant\n{idx2label[sample['label']]}<|im_end|>"
    return sample

train_data = data['train'].map(prepare_data_chatml_format, remove_columns=['label'])
valid_data = data['validation'].map(prepare_data_chatml_format, remove_columns=['label'])
print(train_data[0]['text'])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<|im_start|>user
Identify the sentiment in the text: i didnt feel humiliated <|im_end|>
<|im_start|>assistant
sadness<|im_end|>


# Setting Up Tokenizer, Model, Bits N Bytes Config and PEFT Config

In [5]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True)
peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model.config.use_cache=False
model.config.pretraining_tp=1

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Setting Up Trainer

In [6]:
training_arguments = TrainingArguments(
    output_dir="./logs",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    fp16=True,
    evaluation_strategy="epoch",          
    save_strategy="epoch",
    num_train_epochs=5,
    load_best_model_at_end=True,
    learning_rate=5e-4  
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=valid_data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024
)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
finetuned_model_id = "./tinyllama-finetuned"

trainer.train()

trainer.model.save_pretrained(finetuned_model_id)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.066192
2,1.123800,1.055677
3,1.123800,1.052396
4,1.049900,1.051657
5,1.049900,1.052497


# Merging the LoRA with the base model

In [8]:
pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True
)
peft_model = PeftModel.from_pretrained(
    pretrained_model,
    finetuned_model_id,
    from_transformers=True,
    device_map="auto"
)

model = peft_model.merge_and_unload()

# Inference

In [9]:
generation_config = GenerationConfig(
    penalty_alpha=0.6, do_sample=True, 
    top_k=5,temperature=0.5,repetition_penalty=1.2, 
    max_new_tokens=32, pad_token_id=tokenizer.eos_token_id
)

def generate_response(prompt):
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs, generation_config=generation_config)
        generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        end_idx = generated_response.index('<|im_end|>', len(prompt)) + len('<|im_end|>')
        return generated_response[:end_idx]
    except:
        return ""


In [10]:
def prepare_prompt_chatml_format(sample):
    sample['prompt'] = f"<|im_start|>user\n{PROMPT}{sample['text']}<|im_end|>\n<|im_start|>assistant\n"
    return sample

test_data = data['test'].map(prepare_prompt_chatml_format)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
sample_prompt = test_data[0]['prompt']
print(generate_response(sample_prompt))

<|im_start|>user
Identify the sentiment in the text:im feeling rather rotten so im not very ambitious right now<|im_end|>
<|im_start|>assistant
sadness<|im_end|>


# Evalutation

In [12]:
from tqdm.notebook import trange
from evaluate import load as load_metric

In [13]:
preds = [generate_response(test_data[i]['prompt']) for i in trange(len(test_data))]
references = data['test']['label']

  0%|          | 0/2000 [00:00<?, ?it/s]

In [14]:
predictions = []
for pred in preds:
    try:
        beg_index = pred.index('<|im_start|>assistant') + len('<|im_start|>assistant')
        end_index = pred.index('<|im_end|>', beg_index)
        predictions.append((pred[beg_index:end_index].strip()))
    except:
        predictions.append("")
        
predictions = [label2idx.get(pred, -1) for pred in predictions]

In [15]:
f1_metric = load_metric('f1')

scores = f1_metric.compute(
    references=references, predictions=predictions, average=None
)
for label, score in zip(idx2label, scores['f1'][1:]):
    print(label, score)

sadness 0.8898756660746003
joy 0.8827586206896552
love 0.6976744186046512
anger 0.847863247863248
fear 0.8537735849056605
surprise 0.6964285714285714


In [16]:
scores = f1_metric.compute(
    references=references, predictions=predictions, average='weighted'
)
scores

{'f1': 0.8559185989111698}